In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters langchain-pinecone

  Using cached langchain-0.3.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_upstage-0.3.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_community-0.3.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached SQLAlchemy-2.0.35-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.10.10-cp311-cp311-win_amd64.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.10-py3-none-any.whl.metadata (6.3 kB)
  Using cached langsmith-0.1.135-py3-none-any.whl.metadata (13 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached langchain_openai-0.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any

ERROR: Could not install packages due to an OSError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\LDCC\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\pinecone\\core\\openapi\\shared\\api_client.py'
Check the permissions.


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\LDCC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [19]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large-query")

In [18]:
from langchain_pinecone import PineconeVectorStore

# 데이터를 처음 저장할 때 
index_name = 'tax-upstage-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
# database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

BadRequestError: Bad Request

In [25]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-1-mini-chat")

In [26]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\LDCC\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [28]:
ai_message = qa_chain.invoke({"query": query})

In [29]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 5,760,000원입니다.'}